In [ ]:
import os, glob
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import scipy.stats
import neural_helpers as nh

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Normalize & mask betas 
(hippocampal ROI) (smooth?)

# Pattern analyses

In [ ]:
beh = pd.read_csv('./data/hybrid_data.csv')
# remember to exclude trials from first 6 volumes that don't exist in fmri data